In [1]:
import pandas as pd
import numpy as np

#### Parcel Inputs

In [2]:
parcel_data_path = "C:/Users/etheocharides/Box/Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/PBA50/EIR runs/Baseline Large (s25) runs/NP_v8_FINAL/"
parcel_data_file = "run314_parcel_data_2050.csv"
parcel_data = pd.read_csv(parcel_data_path+parcel_data_file)

In [3]:
# join county col
parcel_xwalk = pd.read_csv("D:/w/bayarea_urbansim/data/2018_10_17_parcel_to_taz1454sub.csv")
parcel_xwalk.PARCEL_ID = parcel_xwalk.PARCEL_ID.round().astype(int)
parcel_data = parcel_data.merge(parcel_xwalk, left_on='parcel_id', right_on='PARCEL_ID', how='left')

In [4]:
print("total number of residential units is {:,}".format(parcel_data.residential_units.sum()))
print("total number of DR units is {:,}".format(parcel_data.deed_restricted_units.sum()))
print("number of preserved units is {:,}".format(parcel_data.preserved_units.sum()))
print("number of subsidized units is {:,}".format(parcel_data.subsidized_units.sum()))
print("number of inclusionary units is {:,}".format(parcel_data.inclusionary_units.sum()))

total number of residential units is 4,254,248.0
total number of DR units is 264,630.0
number of preserved units is 0.0
number of subsidized units is 0.0
number of inclusionary units is 130,329.0


#### Preservation

In [5]:
# preservation units are typically randomly added to buildings 
# they can be added to any unit that is not already deed restricted 

In [6]:
counties = ['ala', 'cnc', 'mar', 'nap', 'sfr', 'smt', 'scl', 'sol', 'son']
num_pres = [22600, 15000, 3150, 1650, 14950, 13500, 28150, 5150, 5900]

for county, num in zip(counties, num_pres):
    parcels_juris = parcel_data.loc[parcel_data.county == county]
    # only add preserved units to parcels with non-DR units
    parcels_for_pres = parcels_juris.loc[(parcels_juris.residential_units) > (parcels_juris.deed_restricted_units)]
    # at the moment only allow ONE UNIT PER PARCEL
    ids_to_pres = np.random.choice(parcels_for_pres.parcel_id, num, replace=False)

    # make units preserved and add to deed restricted total
    parcel_data.loc[parcel_data.parcel_id.isin(ids_to_pres), "preserved_units"] = \
        parcel_data.preserved_units.fillna(0) + 1
    parcel_data.loc[parcel_data.parcel_id.isin(ids_to_pres), "deed_restricted_units"] = \
        parcel_data.deed_restricted_units.fillna(0) + 1

In [7]:
print("total number of residential units is {:,}".format(parcel_data.residential_units.sum()))
print("total number of DR units is {:,}".format(parcel_data.deed_restricted_units.sum()))
print("number of preserved units is {:,}".format(parcel_data.preserved_units.sum()))
print("number of subsidized units is {:,}".format(parcel_data.subsidized_units.sum()))
print("number of inclusionary units is {:,}".format(parcel_data.inclusionary_units.sum()))

total number of residential units is 4,254,248.0
total number of DR units is 374,680.0
number of preserved units is 110,050.0
number of subsidized units is 0.0
number of inclusionary units is 130,329.0


#### Production

In [8]:
# production units are typically added by funding buildings and making them feasible
# here they can be added to parcels with existing buildings (as a "project") 
# or to new buildings (these were feasible, but could have been infeasible in a prior iteration and thus eligible)
# they can be added to any unit that is not already deed restricted 
# NOTE: we likely undercount inclusionary and overcount preservation a bit this way

In [9]:
counties = ['ala', 'cnc', 'mar', 'nap', 'sfr', 'smt', 'scl', 'sol', 'son']
num_prod = [24100, 15900, 3300, 1800, 15900, 14300, 29900, 5400, 6400]

for county, num in zip(counties, num_prod):
    parcels_juris = parcel_data.loc[parcel_data.county == county]
    # only add preserved units to parcels with non-DR units
    parcels_for_prod = parcels_juris.loc[(parcels_juris.residential_units) > (parcels_juris.deed_restricted_units)]
    # at the moment only allow ONE UNIT PER PARCEL
    ids_to_prod = np.random.choice(parcels_for_prod.parcel_id, num, replace=False)

    # make units preserved and add to deed restricted total
    parcel_data.loc[parcel_data.parcel_id.isin(ids_to_prod), "subsidized_units"] = \
        parcel_data.subsidized_units.fillna(0) + 1
    parcel_data.loc[parcel_data.parcel_id.isin(ids_to_prod), "deed_restricted_units"] = \
        parcel_data.deed_restricted_units.fillna(0) + 1

In [10]:
print("total number of residential units is {:,}".format(parcel_data.residential_units.sum()))
print("total number of DR units is {:,}".format(parcel_data.deed_restricted_units.sum()))
print("number of preserved units is {:,}".format(parcel_data.preserved_units.sum()))
print("number of subsidized units is {:,}".format(parcel_data.subsidized_units.sum()))
print("number of inclusionary units is {:,}".format(parcel_data.inclusionary_units.sum()))

total number of residential units is 4,254,248.0
total number of DR units is 491,680.0
number of preserved units is 110,050.0
number of subsidized units is 117,000.0
number of inclusionary units is 130,329.0


#### Output

In [11]:
# save a copy of the data in the output folder, manually make it the master file as needed
# don't want to risk overwriting the master file (runX_parcel_data_2050_AH.csv)
new_parcel_data_file = "run314_parcel_data_2050_add_AH_output.csv"
parcel_data.to_csv(parcel_data_path+new_parcel_data_file)